Codificare una HTTP API con Python


In [ ]:
#main.py
# Import the Flask module that has been installed.
from flask import Flask
from flask import send_file

# Creating a new "app" by using the Flask constructor. Passes __name__ as a parameter.
app = Flask(__name__)

# Annotation that allows the function to be hit at the specific URL.
@app.route("/")
# Generic Python functino that returns "Hello world!"
def index():
    return "Hello world!"

# Checks to see if the name of the package is the run as the main package.
if __name__ == "__main__":
    # Runs the Flask application only if the main.py file is being run.
    app.run()

books.json

In [ ]:
{
  "books": [
    {
      "id": 1,
      "title": "Harry Potter and the Goblet of Fire",
      "author": "J.K. Rowling",
      "isbn": "1512379298"
    },
    {
      "id": 2,
      "title": "Lord of the Flies",
      "author": "William Golding",
      "isbn": "0399501487"
    }
  ]
}

route /books

In [ ]:
# Annotation that allows the function to be hit at the specific URL (/books).
@app.route("/books")
# Generic Python functino that returns books.json
def books():
    return send_file('books.json')

Dependencies per estrarre dati da MongoDB


In [ ]:
pip install Flask-PyMongo
pip install Flask
pip install dnspython # per risolvere l'indirizzo del db
npm install -g --force nodemon
pip install flask-cors

Codice

In [ ]:
# main.py

from flask import Flask
from flask import jsonify
from flask import request
from flask_pymongo import PyMongo
from flask_cors import CORS

app = Flask(__name__)

# Stringa di connessione al DB
app.config["MONGO_URI"] = "**TUO_URL_DI_CONNESSIONE**" #Importante qui va specificato il nome del DB

mongo = PyMongo(app)
# Per rispondere alle chiamate cross origin
CORS(app)

# Annotation that allows the function to be hit at the specific URL.
@app.route("/")
# Generic Python functino that returns "Hello world!"
def index():
    return "Hello world!"

# Questa route effettua una find() su tutto il DB (si limita ai primi 100 risultati)
@app.route('/addresses', methods=['GET'])
def get_all_addresses():
    mil4326WKT = mongo.db.MilWKT4326
    output = []
    for s in mil4326WKT.find().limit(100):
        output.append(s['INDIRIZZO'])
    return jsonify({'result': output})

# Checks to see if the name of the package is the run as the main package.
if __name__ == "__main__":
    # Runs the Flask application only if the main.py file is being run.
    app.run()


Visualizzare tutti i vettori

In [ ]:
@app.route('/ci_vettore', methods=['GET'])
def get_vettore():
    mil4326WKT = mongo.db.MilWKT4326
    output = []
    for s in mil4326WKT.find().limit(100):
        output.append(s['CI_VETTORE'])
    return jsonify({'result': output})

Visualizzare in base ad un parametro

In [ ]:
@app.route('/ci_vettore/<foglio>', methods=['GET'])
def get_vettore(foglio):
    mil4326WKT = mongo.db.MilWKT4326
    output = []
    query = {
        "FOGLIO" : foglio
    }
    for s in mil4326WKT.find(query).limit(100):
        output.append(s['CI_VETTORE'])
    return jsonify({'result': output})

recuperare tutti i dati risultati dalla query e formattarli in JSON

In [ ]:
@app.route('/ci_vettore/<foglio>', methods=['GET'])
def get_vettore(foglio):
    mil4326WKT = mongo.db.MilWKT4326
    output = []
    query = {
        "FOGLIO" : foglio
    }
    for s in mil4326WKT.find(query):
        output.append({
            "INDIRIZZO":s['INDIRIZZO'],
            "WGS84_X":s["WGS84_X"],
            "WGS84_Y":s["WGS84_Y"],
            "CLASSE_ENE":s["CLASSE_ENE"],
            "EP_H_ND":s["EP_H_ND"],
            "FOGLIO":s["FOGLIO"],
            "CI_VETTORE":s['CI_VETTORE']
        }
        )
    return jsonify(output) #Nota che abbiamo eliminato la chiave result perchè i dati sono già formattati

Aggregazioni

In [ ]:
@app.route('/geogeom', methods=['GET'])
def get_all_stars():
    mil4326WKT = mongo.db.MilWKT4326
    output = []

    group = {
        '$group': {
            '_id': {
                'SEZ': '$SEZ',
                'WKT': '$WKT'
            },
            'AVG': {
                '$avg': '$EP_H_ND'
            },
            'SUM': {
                '$sum': '$EP_H_ND'
            }
        }
    }

    for s in mil4326WKT.aggregate([group]):
        output.append({'somma': s['SUM'], 'media': s['AVG'],
                      'WKT': s['_id']['WKT'], 'SEZ': s['_id']['SEZ']})
    return jsonify({'result': output})

Trasformare i dati in GeoJson


In [ ]:
import geojson
import shapely.wkt

In [ ]:
@app.route('/geogeom', methods=['GET'])
def get_all_stars():
    mil4326WKT = mongo.db.MilWKT4326
    output = []

    match = {
        '$match': {
            'EP_H_ND': {'$gt': 0}
        }
    }
    group = {
        '$group': {
            '_id': {
                'SEZ': '$SEZ',
                'WKT': '$WKT'
            },
            'AVG': {
                '$avg': '$EP_H_ND'
            },
            'SUM': {
                '$sum': '$EP_H_ND'
            }
        }
    }
    limit = {
        '$limit' : 10
    }
    
    for s in mil4326WKT.aggregate([match, group, limit]):
        g1= shapely.wkt.loads(s['_id']['WKT'])
        g2 = geojson.Feature(geometry=g1, 
        properties={'id':s['_id']['SEZ'], 'media':s['AVG'], 'somma':s['SUM'], 'sezione':s['_id']['SEZ']}) 
        output.append(g2)                 
    return jsonify({'result': output})

Mettendo assieme le due query, possiamo completare il codice come segue

In [ ]:

# Estrae consumo energetico medio e totale raggruppato per sezione catastale (e Poligono WKT)
@app.route('/geogeom/<float:lng>/<float:lat>/<float:r>', methods=['GET'])
def get_avg(lng, lat, r):
    print(type(lng))
    mil4326WKT = mongo.db.MilWKT4326
    output = []

    match = {
        '$match': {
            '$and':
            [
                {'EP_H_ND': {'$gt': 0}},
                {'WGS84_X': {'$gt': lng - r}},
                {'WGS84_X': {'$lt': lng + r}},
                {'WGS84_Y': {'$gt': lat - r}},
                {'WGS84_Y': {'$lt': lat + r}}
            ]
        }
    }
    group = {
        '$group': {
            '_id': {
                'SEZ': '$SEZ',
                'WKT': '$WKT'
            },
            'AVG': {
                '$avg': '$EP_H_ND'
            },
            'SUM': {
                '$sum': '$EP_H_ND'
            }
        }
    }
    limit = {
        '$limit': 10
    }

    for s in mil4326WKT.aggregate([match, group, limit]):
        # Converte da WKT in GeoJson Geometry
        g1 = shapely.wkt.loads(s['_id']['WKT'])
        g2 = geojson.Feature(geometry=g1,
                             properties={'id': s['_id']['SEZ'], 'media': s['AVG'], 'somma': s['SUM'], 'sezione': s['_id']['SEZ']})
        output.append(g2)
    return jsonify(geojson.FeatureCollection(output))